In [57]:
import pyodbc 
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("./Player.2024.csv")

print(df.head())
print(df.shape)

# TODO: https://www.dropbox.com/scl/fi/hy0sxw6gaai7ghemrshi8/lahman_1871-2023_csv.7z?dl=0&e=1&file_subpath=%2Flahman_1871-2023_csv&rlkey=edw1u63zzxg48gvpcmr3qpnhz

   PlayerID    Status  TeamID Team  Jersey PositionCategory Position  \
0  10000001  Inactive     NaN  NaN     NaN                P       RP   
1  10000004  Inactive     NaN  NaN     NaN                P       SP   
2  10000005  Inactive     NaN  NaN     NaN                P       RP   
3  10000007  Inactive     NaN  NaN     NaN                P       SP   
4  10000008  Inactive     NaN  NaN     NaN                P       RP   

  FirstName    LastName BatHand  ...               BirthDate      BirthCity  \
0     Chase    Anderson       R  ...  11/30/1987 12:00:00 AM  Wichita Falls   
1    Trevor      Cahill       R  ...    3/1/1988 12:00:00 AM      Oceanside   
2    Andrew      Chafin       R  ...   6/17/1990 12:00:00 AM      Kettering   
3   Patrick      Corbin       L  ...   7/19/1989 12:00:00 AM           Clay   
4     Rubby  De La Rosa       R  ...    3/4/1989 12:00:00 AM  Santo Domingo   

   BirthState        BirthCountry  \
0          TX                 USA   
1          CA     

In [ ]:
server = 'tcp:sports-iq.database.windows.net,1433' 
database = 'main' 
username = 'CloudSAeab82b26' 
password = '' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [59]:

df = df.dropna(subset=['PositionCategory'])
df = df.replace(np.nan, None)

print(type(df['BirthDate']))

print(df.shape)

<class 'pandas.core.series.Series'>
(8664, 23)


In [66]:
for index, row in df.iterrows():
		
		cursor.execute("SELECT t.TeamID FROM [MLB].[Teams] t WHERE Abbreviation = ?", row.Team)
		team = cursor.fetchone()

		playerTeam = int(team[0]) if team is not None else None

		date = None

		if (row.BirthDate is not None):
			date = datetime.strptime(row.BirthDate, "%m/%d/%Y %H:%M:%S %p")
																																																																																												#11/30/1987 12:00:00 AM
		cursor.execute("INSERT INTO MLB.Players (FirstName,LastName,Jersey,PositionCategory,Position,Status,TeamID, DateOfBirth) VALUES(?,?,?,?,?,?,?,?)", row.FirstName, row.LastName, row.Jersey, row.PositionCategory, row.Position, row.Status, playerTeam, date)

cnxn.commit()
cursor.close()